<a href="https://colab.research.google.com/github/LeMikey/SDGP-HelloWorld/blob/main/main_Landslide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Landslide Prediction System**

**Imports and installation**

In [ ]:
!pip install pytorch_tabnet 

In [13]:
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score
np.random.seed(0)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.impute import SimpleImputer

import torch

from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import KFold
import torch

%matplotlib inline

**Dataset collection and altering**

In [4]:
trainSet = pd.read_csv('train-landslide-dataset.csv')
testSet = pd.read_csv('test-landslide-dataset.csv')

trainSet['Landslide'] = trainSet['Landslide'].astype(str)

features = [col for col in trainSet.columns if col not in ['Landslide']]

pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=200, output_distribution='normal'))
        ])
X = pipe.fit_transform(trainSet[features])
X_test=pipe.transform(testSet[features])

In [15]:
trainSet.shape

(1212, 13)

# Machine Learning Model

In [16]:
tabnet_params = dict(n_steps = 1,
                   optimizer_fn=torch.optim.Adam,
                   optimizer_params=dict(lr=1e-2, weight_decay = 5e-4),
                   scheduler_params={"step_size":1,
                                     "gamma":0.9},
                   scheduler_fn=torch.optim.lr_scheduler.StepLR,
                   mask_type='entmax',
                   verbose = 5)

kf = KFold(n_splits=5, random_state = 40, shuffle = True)
preds = np.zeros((243,))
for  fold , (train_index, test_index) in enumerate(kf.split(X)):
    print(20*"*")
    print("Fold {}:".format(fold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = trainSet.Landslide[train_index].values, trainSet.Landslide[test_index].values

    clf = TabNetClassifier(**tabnet_params)
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs= 100, patience=5,
        batch_size=1024*10, virtual_batch_size=128*10,
        num_workers=0,
        weights=1,
        drop_last=False)
     
    preds += clf.predict_proba(X_test)[:, 1]
    print(preds.shape)

********************
Fold 0:
Device used : cpu
epoch 0  | loss: 0.69087 | train_auc: 0.53123 | valid_auc: 0.50173 |  0:00:00s
epoch 5  | loss: 0.62337 | train_auc: 0.57789 | valid_auc: 0.55935 |  0:00:00s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_valid_auc = 0.57655
Best weights from best epoch are automatically used!


ValueError: ignored

In [ ]:
submissionData = pd.read_csv('submission-file.csv')
submissionData